<a href="https://colab.research.google.com/github/santanaemmanuel/redes_neurais_CESAR_SCHOOL/blob/main/Redes_Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing
Esta primeira etapa tem como objetivo realizar o download do dataset e prepará-lo em um grupo de treino e outro de testes.

In [ ]:
import zipfile
import os

#O dataset a ser utilizado foi carregado no github
!wget --no-check-certificate \
    "https://github.com/santanaemmanuel/redes_neurais_CESAR_SCHOOL/archive/refs/heads/main.zip" \
    -O "/content/mtg-cards.zip"

#Realiza o dowload e a descompactação do dataset a ser utilizado
zip_ref = zipfile.ZipFile('/content/mtg-cards.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /content folder
zip_ref.close()


In [ ]:
!git clone https://github.com/vcasadei/cats-and-dogs.git

Uma vez que o dataset foi carregado no notebook, o próximo passo será dividí-lo em grupos de treino e teste

In [ ]:
import os
import shutil
import re

dataset_dir = "/content/redes_neurais_CESAR_SCHOOL-main/dataset/mtg_cards"
base_dir = "mtg-cards"


# Moves all training cat images to cats folder, training dog images to dogs folder
def val_train_maker(name, val_size):
  try:
      path_train = f"{base_dir}/train/{name}"
      path_val = f"{base_dir}/val/{name}"
      os.makedirs(path_train, exist_ok=True)
      os.makedirs(path_val, exist_ok=True)
  except OSError:
      print ("Creation of the directory failed")
  else:
      print ("Successfully created the directories")
  files_path = os.path.join(dataset_dir, name)
  files = os.listdir(files_path)
  file_count = 0
  for f in files:
    if file_count < val_size:
      shutil.move(f'{files_path}/{f}', path_train)
      file_count += 1
    else:
      shutil.move(f'{files_path}/{f}', path_val)

class_names = ['W','B','R','G','U']
for name in class_names:
   val_train_maker(name, 120)

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy

In [ ]:
# Make transforms and use data loaders

# We'll use these a lot, so make them variables
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]), 'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean_nums, std_nums)
]),
}

In [ ]:
# Set the directory for the data
data_dir = 'PetImages/'

# Use the image folder function to create datasets
chosen_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), chosen_transforms[x]) for x in ['train', 'val']}

In [ ]:
# Make iterables with the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=4,
  shuffle=True, num_workers=4)
              for x in ['train', 'val']}

In [ ]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'val']}
class_names = chosen_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(dataset_sizes)
print(class_names)

#Modelo
O modelo resnet utilizado será o não treinado.

In [ ]:
# Setting up the model
# load in pretrained and reset final fully connected

res_mod = models.resnet34(pretrained=False)

num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 2)

In [ ]:
res_mod = res_mod.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)